# Stability Analysis of AdImEx Advection without splitting

## Discretisation of the Advection Equation
\begin{eqnarray}
\frac{\partial \psi}{\partial t} &=& -u \frac{\partial \psi}{\partial x} \\
\text{Assume } \psi_j &=& e^{ikj\Delta x},\ c = u\Delta t/\Delta x,\ \lambda = u/\Delta x \\
\end{eqnarray}
\begin{eqnarray}
\text{Analytic } \frac{dy}{dt} &=& - i\lambda k\Delta x y \\
\end{eqnarray}
The spatially discretised advection equation can be written as a correction on upwind:
\begin{eqnarray}
\frac{dy}{dt} &=& -\lambda(\mu  + \eta) y\\
\text{where } \mu &=& 1-\cos k\Delta x + i\sin k\Delta x \ \text{ (upwind)}\\
\text{and }   \eta &=& -\mu + \left\{
\sum_\ell w_\ell\  e^{i\ell k\Delta x} - \sum_{\ell-1} w_\ell\  e^{i\ell k\Delta x}
\right\}  \ \text{ (correction)}
\end{eqnarray}
where $w_\ell$ are the high-order correction weights over cells indexed by $\ell$.

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sympy as sy
from sympy import latex
from sympy import I, E, pi, Q
import os

In [ ]:
# mu and eta for a scheme with given weights
kDx = sy.symbols("kΔx", real=True)
def muEta(indicies, weights):
    mu = 1 - E**(-I*kDx)
    eta = -mu
    for j,w in zip(indicies, weights):
        eta += w*(E**(j*I*kDx) - E**((j-1)*I*kDx))
    return mu, eta

# Quasi-cubic
indicies3 = [-1,0,1]
weights3 = [sy.Rational(-1,6), sy.Rational(5,6), sy.Rational(1,3)]
mu, eta3 = muEta(indicies3, weights3)

# Quasi-quartic
indicies4 = [-2,-1,0,1]
weights4 = [sy.Rational(1,12), sy.Rational(-5/12), sy.Rational(13/12), sy.Rational(1/4)]
mu, eta4 = muEta(indicies4, weights4)

# Quasi-quintic
indicies5 = [-2,-1,0,1,2]
weights5 = [sy.Rational(1,30), sy.Rational(-13,60), sy.Rational(47/60), sy.Rational(9/20), 
           sy.Rational(-1/20)]
mu, eta5 = muEta(indicies5, weights5)

# First-order upwind
eta1 = 0

In [ ]:
# RK3, RK4 and RK1 Butcher Tableau
quarter = sy.Rational(1,4)
sixth = sy.Rational(1,6)
half  = sy.Rational(1,2)
third = sy.Rational(1,3)
RK3 = [[1,0,0], [quarter,quarter,0], [sixth, sixth, 4*sixth]]
RK4 = [[half,0,0,0], [0, half,0,0], [0,0,1,0], [sixth, third, third, sixth]]
RK1 = [[1]]

In [ ]:
# Amplification factors for the unsplit AdImEx scheme for a given RK scheme
# and a given spatial discretistaion (defined by mu and eta)
# as a function of Courant number, c and parameters alpha, beta and gamma
c, alpha, beta, gamma = sy.symbols("c, alpha, beta, gamma", real=True, positive=True)
def ampFactor(RK, mu, eta):
    RKc = sy.Matrix(sy.symarray("A", (len(RK),)))
    for i in range(len(RK)):
        RKc[i] = 0
        for j in range(0,i+1):
            RKc[i] += RK[i][j]

    A = sy.Matrix(sy.symarray("A", (len(RK)+1,)))
    A[0] = 1
    for i in range(len(RK)):
        A[i+1] = 1 - c*RKc[i]*(1-alpha)*beta*mu
        for j in range(0,i+1):
            A[i+1] -= c*RK[i][j]*((1-beta)*mu + gamma*eta)*A[j]
        A[i+1] /= (1 + c*RKc[i]*alpha*beta*mu)
    return A[-1]

# Find the maximum amplification factor over a range of kDx for a given c, alpha,
# beta, gamma
def maxMagA(A, c_, alpha_, beta_, gamma_, kdxs = np.linspace(np.pi/4, np.pi, 10)):
    magA = 0
    for kdx in kdxs:
        Atmp = A.subs({alpha: alpha_, beta: beta_, gamma: gamma_, kDx: kdx, c: c_})
        magA = max(magA, abs(Atmp.evalf()))
    return magA

In [ ]:
# Amplification factors for specific schemes
A33 = ampFactor(RK3, mu, eta3)
A34 = ampFactor(RK4, mu, eta3)
A53 = ampFactor(RK3, mu, eta5)
A54 = ampFactor(RK4, mu, eta5)
A43 = ampFactor(RK3, mu, eta4)
A44 = ampFactor(RK4, mu, eta4)
A11 = ampFactor(RK1, mu, eta1)

In [ ]:
# Plots of |A| for various c, kdx, for alpha=0.5, beta=0, gamma=1
alpha_ = 0.5; beta_ = 0.; gamma_ = 1.
As = [A33, A54]
titles=["Cubic, RK3", "Quintic, RK4"]
cmaxs = [1.6, 1.7]
outFiles=["magAe_33.pdf", "magAe_54.pdf"]
kdxs = np.linspace(0, np.pi, 40)
cs = np.linspace(0, 2, 41)
for A, title, cmax, outFile in zip(As, titles, cmaxs, outFiles):
    magA = np.zeros([len(kdxs), len(cs)])
    for ic in range(len(cs)):
        c_ = cs[ic]
        for ik in range(len(kdxs)):
            kdx = kdxs[ik]
            Atmp = A.subs({alpha: alpha_, beta: beta_, gamma: gamma_, c: c_, kDx: kdx})
            magA[ik,ic] = abs(Atmp.evalf())

    # Plot magA
    ax = plt.contourf(cs, kdxs, magA, levels=np.linspace(0,2,21), extend='max', cmap='seismic')
    plt.title(title)
    plt.colorbar(ax)
    plt.contour(cs, kdxs, magA, [-1,1], colors='k')
    plt.axvline(x=cmax, ls=':', color='k', label=r'$c_\max =$'+str(cmax))
    plt.legend()
    plt.xlabel(r'$c$')
    plt.ylabel(r'$k\Delta x$')
    plt.savefig('plots/'+outFile)
    os.system('pdfCrop plots/'+outFile)
    plt.show()

In [ ]:
# Calculate maxMagA for various c and beta for alpha=1, gamma=1
alpha_ = 1; gamma_ = 1.
As = [A33, A54]
magAs = []
# Non-linear range of Courant numbers
cs = 0.1*10**np.linspace(0,3,37)
betas = np.linspace(0, 1, 41)
for A in As:
    magA = np.zeros([len(betas), len(cs)])
    for ic in range(len(cs)):
        c_ = cs[ic]
        for ib in range(len(betas)):
            beta_ = betas[ib]
            magA[ib,ic] = maxMagA(A, c_, alpha_, beta_, gamma_, 
                                  kdxs = np.linspace(np.pi/2, np.pi, 21))
    magAs.append(magA)

In [ ]:
# Plots these maxMagA
titles=["Cubic, RK3", "Quintic, RK4"]
cmaxs = [1.6, 1.7]
outFiles=["maxMagAbeta_33.pdf", "maxMagAbeta_54.pdf"]
def beta33(c):
    cmax = 1.6; g = 0.35/(3-cmax); a = 0.41; b = a**2/g; d = cmax - b/a
    print('beta33: g =', g, 'b =', b, 'd =', d)
    return a - b/(c-d)

def beta54(c):
    cmax = 1.7; g = 0.35/(3-cmax); a = 0.43; b = a**2/g; d = cmax - b/a
    print('beta54: g =', g, 'b =', b, 'd =', d)
    return a - b/(c-d)

def betaMax(c):
    cmax = 1.5; g = 0.35/(3-cmax); a = 0.5; b = a**2/g; d = cmax - b/a
    print('betaMax: g =', g, 'b =', b, 'd =', d)
    return a - b/(c-d)

for magA, title, cmax, b, outFile in zip(magAs, titles, cmaxs, [beta33, beta54], outFiles):
    # Plot magA
    fig, ax1 = plt.subplots()
    ax = plt.contourf(cs, betas, magA, levels=np.linspace(0,2,21), extend='max', 
                      cmap='seismic')
    ax1.set_xscale('log')
    plt.title(title)
    plt.colorbar(ax)
    plt.contour(cs, betas, magA, [-1,1], colors='k')
    plt.plot(cs, b(cs), 'g--', label=r'$\beta=0.4 - b/(c-d)$')
    plt.plot(cs, betaMax(cs), 'g--', label=r'$\beta=0.5 - b/(c-d)$')
    plt.axvline(x=cmax, ls=':', color='k', label=r'$c_\max =$'+str(cmax))
    plt.legend()
    plt.ylim([0,0.5])
    plt.xlim([1,cs[-1]])
    plt.xlabel(r'$c$')
    plt.ylabel(r'$\beta$')
    plt.savefig('plots/'+outFile)
    os.system('pdfCrop plots/'+outFile)
    plt.show()

In [ ]:
# Calculate maxMagA for various c and beta for alpha=1-1/max(c,2), gamma=1
def alphaC(c):
    return 1 - 1/max(c,2)
gamma_ = 1.
Aas = [A33, A54]
magAas = []
# Non-linear range of Courant numbers
cs = 10**np.linspace(0,1,21)
betas = np.linspace(0, 0.5, 21)
for A in Aas:
    magA = np.zeros([len(betas), len(cs)])
    for ic in range(len(cs)):
        c_ = cs[ic]
        for ib in range(len(betas)):
            beta_ = betas[ib]
            magA[ib,ic] = maxMagA(A, c_, alphaC(c_), beta_, gamma_, 
                                  kdxs = np.linspace(np.pi/2, np.pi, 21))
    magAas.append(magA)

In [ ]:
# Plots these maxMagA
titles=["Cubic, RK3", "Quintic, RK4"]
legends=['\\beta_{33}','\\beta_{54}']
cmaxs = [1.6, 1.7]
outFiles=["maxMagAabeta_33.pdf", "maxMagAabeta_54.pdf"]
def beta33(c):
    cmax = 1.6; g = 0.6/(3-cmax); a = 0.43; b = a**2/g; d = cmax - b/a
    print('beta33: g =', g, 'b =', b, 'd =', d)
    return a - b/(c-d)

def beta54(c):
    cmax = 1.7; g = 0.5/(3-cmax); a = 0.45; b = a**2/g; d = cmax - b/a
    print('beta54: g =', g, 'b =', b, 'd =', d)
    return a - b/(c-d)

for magA, title, cmax, b, outFile, leg in \
        zip(magAas, titles, cmaxs, [beta33, beta54], outFiles, legends):
    # Plot magA
    fig, ax1 = plt.subplots()
    ax = plt.contourf(cs, betas, magA, levels=np.linspace(0,2,21), extend='max', 
                      cmap='seismic')
    ax1.set_xscale('log')
    plt.title(title)
    plt.colorbar(ax)
    plt.contour(cs, betas, magA, [-1,1], colors='k')
    plt.plot(cs, b(cs), 'g--', label=r'$'+leg+'$')
    plt.axvline(x=cmax, ls=':', color='k', label=r'$c_\max =$'+str(cmax))
    plt.legend()
    plt.ylim([0,betas[-1]])
    plt.xlim([1,cs[-1]])
    plt.xlabel(r'$c$')
    plt.ylabel(r'$\beta$')
    plt.savefig('plots/'+outFile)
    os.system('pdfCrop plots/'+outFile)
    plt.show()